# Notebook Goals

1. The notebook will look at households that the dataset provides transaction and demographic data for. 

2. From there we will breakout the households out by age groups and income group. 
 
3. Chart the spending patterns of two specific households during the active adveristing campaigns.

4. Plot Average Weekly Spending for demographic groups Age Group (19-24) and Income Group 50-74K.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
campaign_desc = pd.read_csv('/kaggle/input/dunnhumby-the-complete-journey/campaign_desc.csv')
product = pd.read_csv('/kaggle/input/dunnhumby-the-complete-journey/product.csv')
coupon_redempt = pd.read_csv('/kaggle/input/dunnhumby-the-complete-journey/coupon_redempt.csv')
transaction_data = pd.read_csv('/kaggle/input/dunnhumby-the-complete-journey/transaction_data.csv')
hh_demographic = pd.read_csv('/kaggle/input/dunnhumby-the-complete-journey/hh_demographic.csv')
coupon = pd.read_csv('/kaggle/input/dunnhumby-the-complete-journey/coupon.csv')
campaign_table = pd.read_csv('/kaggle/input/dunnhumby-the-complete-journey/campaign_table.csv')
causal_data = pd.read_csv('/kaggle/input/dunnhumby-the-complete-journey/causal_data.csv')

# 1. Finding households that have transaction data and demographic data available.

In [ ]:
#  Get a list of all unique household keys in transaction_data
trans_data_hh_key_list = list(transaction_data.household_key.unique())

#  Get a list of all unique household keys in hh_demoographics
hh_demo_hh_key_list = list(hh_demographic.household_key.unique())

In [ ]:
## A function to find common items in two lists

def common_member(a, b): 
    """A function to find the common items in two lists."""
    a_set = set(a) 
    b_set = set(b) 
    if (a_set & b_set): 
        return list((a_set & b_set))
    else: 
        print("No common elements")  

In [ ]:
#  Create a list of unique household keys that transaction data AND demographic data is avaliable for.
trans_data_and_hh_demo_hh_keys_list = common_member(trans_data_hh_key_list, hh_demo_hh_key_list)

#  Print the length of the list so we know the amount of household we have avaliable data for
print("We have transaction AND demongraphic data on " + str(len(trans_data_and_hh_demo_hh_keys_list)) + " households.")

## 1.a Demographic Info

In [ ]:
hh_demographic.head()

## 1.b Transaction Data

Looking at the transaction data, we see that each row represents a specific product.

We can group the data by BASKET_ID as the transaction time, day, and week number are the same for each basket of products. From there we can sum each baskets total sales value to get the sales total for each basket of goods.**

In [ ]:
transaction_data

### Group transaction data by basket for each household_key and SUM each basket of transactions

In [ ]:
trans_data_baskets_hh_key = pd.DataFrame(transaction_data.groupby(['household_key', 'BASKET_ID']).sum()).drop(['DAY', 'PRODUCT_ID','QUANTITY', 'STORE_ID', 'RETAIL_DISC', 'TRANS_TIME', 'WEEK_NO', 'COUPON_DISC', 'COUPON_MATCH_DISC'], axis=1)
trans_data_baskets_hh_key = trans_data_baskets_hh_key.merge(transaction_data.drop(['SALES_VALUE', 'PRODUCT_ID', 'QUANTITY', 'RETAIL_DISC', 'COUPON_DISC', 'COUPON_MATCH_DISC'], axis=1), on="BASKET_ID").drop_duplicates(subset=['BASKET_ID'])
trans_data_baskets_hh_key

## Merge the demographic Data to each transaction basket sales data.

Because the transaction data provides a household key associated with each product ID and basket ID, we can merge the transaction with the demographic data.

In [ ]:
spend_demo_data = trans_data_baskets_hh_key.merge(hh_demographic, on='household_key')
spend_demo_data.head()

# 2. Show Transaction Data by Age Group and Income Group

## 2.a Show Transaction Data by Age Group

In [ ]:
#  Get the unique age_groups in AGE_DSC
age_groups = sorted(list(spend_demo_data.AGE_DESC.unique()))
print("The different age_group buckets in the dataset are " + str(age_groups) + ".")

# Create empty list of to store dataframes
age_group_spending_dfs = []

# append each individual age group dataframe to the list
for group in age_groups:
    age_group_spending_dfs.append(spend_demo_data[spend_demo_data['AGE_DESC'] == group])
    
    
# Group and Name the Transaction Spending DataFrames by Age Group
age_19_24 = age_group_spending_dfs[0]
age_25_34 = age_group_spending_dfs[1]
age_35_44 = age_group_spending_dfs[2]
age_45_54 = age_group_spending_dfs[3]
age_55_64 = age_group_spending_dfs[4]
age_65 = age_group_spending_dfs[5]

## Plotting Transaction Spend by Specific Households Across Time With Campaign

### Matching Campaigns to Specific Households

In [ ]:
hh_key_48 = age_19_24[age_19_24['household_key'] == 48]
hh_key_48_campaigns = campaign_table[campaign_table['household_key'] == 48]
hh_key_48_campaigns

In [ ]:
campaign_desc['campaign_length_days'] = campaign_desc['END_DAY'] - campaign_desc['START_DAY']
print(campaign_desc[campaign_desc['CAMPAIGN'] == 18])
print(campaign_desc[campaign_desc['CAMPAIGN'] == 19])

## 2.b Transaction Data by Income Group

In [ ]:
#  Get the unique age_groups in AGE_DSC
income_groups = sorted(list(spend_demo_data.INCOME_DESC.unique()))
print("The different income buckets in the dataset are " + str(income_groups) + ".")

# Create empty list of to store dataframes
income_group_spending_dfs = []

# append each individual age group dataframe to the list
for group in income_groups:
    income_group_spending_dfs.append(spend_demo_data[spend_demo_data['INCOME_DESC'] == group])
    

#  Group and Name the Transaction Spending DataFrames by Income Group
inc_100_124K = income_group_spending_dfs[0]
inc_125_149K = income_group_spending_dfs[1]
inc_15_24K = income_group_spending_dfs[2]
inc_150_174K = income_group_spending_dfs[3]
inc_175_199K = income_group_spending_dfs[4]
inc_200_249K = income_group_spending_dfs[5]
inc_25_34K = income_group_spending_dfs[6]
inc_250K = income_group_spending_dfs[7]
inc_35_49K = income_group_spending_dfs[8]
inc_50_74K = income_group_spending_dfs[9]
inc_75_99K = income_group_spending_dfs[10]
inc_Under_15K = income_group_spending_dfs[11]

### Matching Campaigns to Specific Households

In [ ]:
hh_key_7 = inc_50_74K[inc_50_74K['household_key'] == 7]
hh_key_7_campaigns = campaign_table[campaign_table['household_key'] == 7]
hh_key_7_campaigns

In [ ]:
print(campaign_desc[campaign_desc['CAMPAIGN'] == 9])
print(campaign_desc[campaign_desc['CAMPAIGN'] == 10])
print(campaign_desc[campaign_desc['CAMPAIGN'] == 16])
print(campaign_desc[campaign_desc['CAMPAIGN'] == 18])

# 3. Plotting Spending Patterns of the Two Specific Household During Active Campaigns

## Plotting transaction basket sales values for household 48 with active campaign times 

In [ ]:
sns.set_style("darkgrid")

plt.figure(figsize=(20,10))
plt.title("Transaction Basket Sales Value by Day", size=26)

campaign_18 = np.arange(587,643)
campaign_19 = np.arange(603,636)
y = hh_key_48.SALES_VALUE.max()
ax = sns.lineplot(x='DAY', y='SALES_VALUE',
                  data=hh_key_48,
                  markers=True)
ax.fill_between(campaign_18, y, facecolor='red', alpha=0.1, label='campaign_18')
ax.fill_between(campaign_19, y, facecolor='green', alpha=0.2, label='campaign_19')
ax.set_xlabel('Day', fontsize=22)
ax.set_ylabel('Sales Value ($)', fontsize=22)
ax.tick_params(axis="x", labelsize=18)
ax.tick_params(axis="y", labelsize=18)
ax.legend()

## Plotting transaction basket sales values for household 7 with active campaign times 

In [ ]:
sns.set_style("darkgrid")

plt.figure(figsize=(20,10))
plt.title("Transaction Basket Sales Value by Day", size=26)

campaign_9 = np.arange(435,468)
campaign_10 = np.arange(463,496)
campaign_16 = np.arange(561,594)
campaign_18 = np.arange(587,643)
y = hh_key_7.SALES_VALUE.max()

ax = sns.lineplot(x='DAY', y='SALES_VALUE',
                  data=hh_key_7,
                  markers=True)
ax.fill_between(campaign_9, y, facecolor='red', alpha=0.1, label='campaign_9')
ax.fill_between(campaign_10, y, facecolor='green', alpha=0.2, label='campaign_10')
ax.fill_between(campaign_16, y, facecolor='orange', alpha=0.3, label='campaign_16')
ax.fill_between(campaign_18, y, facecolor='purple', alpha=0.4, label='campaign_18')

ax.set_xlabel('Day', fontsize=22)
ax.set_ylabel('Sales Value ($)', fontsize=22)
ax.tick_params(axis="x", labelsize=18)
ax.tick_params(axis="y", labelsize=18)
plt.legend()

# 4. Average Weekly Spending by Age Group (19-24) and Income Group 50-74K

In [ ]:
campaign_desc['START_WEEK'] = campaign_desc['START_DAY'] / 7
campaign_desc['END_WEEK'] = campaign_desc['END_DAY'] / 7
campaign_desc.head()

In [ ]:
household_spending_19_24 = age_19_24[['WEEK_NO', 'SALES_VALUE', 'household_key']].groupby(['WEEK_NO', 'household_key']).mean().reset_index()
plt.figure(figsize=(20,10))
ax = sns.lineplot(x='WEEK_NO', y='SALES_VALUE',
                  data=household_spending_19_24,
                  markers=True)

In [ ]:
sns.set_style("darkgrid")

avg_daily_sales_19_24 = age_19_24[['WEEK_NO', 'SALES_VALUE', 'household_key']].groupby(['WEEK_NO', 'household_key']).mean().reset_index()
plt.figure(figsize=(20,10))
plt.title("Average Transaction Basket Sales Value by Week for Age Group 19-24", size=26)
ax = sns.lineplot(x='WEEK_NO', y='SALES_VALUE',
                  data=avg_daily_sales_19_24,
                  markers=True)

In [ ]:
sns.set_style("darkgrid")

avg_daily_sales_inc_50_74K = inc_50_74K[['WEEK_NO', 'SALES_VALUE', 'household_key']].groupby(['WEEK_NO', 'household_key']).mean().reset_index()
plt.figure(figsize=(20,10))
plt.title("Average Transaction Basket Sales Value by Week for Income Group $50-74K", size=26)
ax = sns.lineplot(x='WEEK_NO', y='SALES_VALUE',
                  data=avg_daily_sales_inc_50_74K,
                  markers=True)